In [2]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from glob import glob 
import tqdm as tqdm

Using TensorFlow backend.


In [0]:
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

In [4]:

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#lists the content of your google drive
!ls "/content/drive/My Drive"

 143645.gdoc
 143645.pdf
 1523514756192_E-commerce-qb.gdoc
'26-12-III B.Tech. II Semester (R13) Regular.xls'
'26-12-III B.Tech. II Semester (R13) Regular.xls.gsheet'
 33244
'33244 (1).gsheet'
'33244 (2).gsheet'
 33244.gsheet
'ACD Text.pdf'
 Airplane_car_data.zip
'Allotments- MGITMUN 2017.gsheet'
'ANKAM MANITEJ resume (1).docx'
'ANKAM MANITEJ resume (1).docx.gdoc'
'ANKAM MANITEJ resume.docx'
'ANKAM MANITEJ resume.docx.gdoc'
'ANKAM MANITEJ resume.gdoc'
'ANKAM MANITEJ resume.pdf'
'Auto Call Recorder'
 Bank.gdoc
'Block Chain TECH.pptx'
'Block Chain TECH.pptx.gslides'
 bottleneck_features
'CERTIFICATE DOC.gdoc'
 ChangeMaker.java
'class diagram for ATM System.png'
'Colab Notebooks'
'COVER LETTER.pdf'
 cv5.apk
 Cv6.apk
 dogImages
 dogImages2
'Fourier Transforms.pdf'
 helloworld.rtf
 helloworld.rtf.gdoc
'How to RUN'
 IMG-20171103-WA0020.jpg
 jess301.pdf
 jess302.pdf
 jess303.pdf
'jess304 (1).pdf'
 jess304.pdf
 jess305.pdf
 jess306.pdf
 jess307.pdf
 jess308.pdf
 jess3ps.pdf
'Manitej COVER LETTE

In [6]:
train_files, train_targets = load_dataset('/content/drive/My Drive/dogImages/train')
valid_files, valid_targets = load_dataset('/content/drive/My Drive/dogImages/valid')
test_files, test_targets = load_dataset('/content/drive/My Drive/dogImages/test')
print(train_files)

['/content/drive/My Drive/dogImages/train/095.Kuvasz/Kuvasz_06442.jpg'
 '/content/drive/My Drive/dogImages/train/057.Dalmatian/Dalmatian_04054.jpg'
 '/content/drive/My Drive/dogImages/train/088.Irish_water_spaniel/Irish_water_spaniel_06014.jpg'
 ...
 '/content/drive/My Drive/dogImages/train/029.Border_collie/Border_collie_02069.jpg'
 '/content/drive/My Drive/dogImages/train/046.Cavalier_king_charles_spaniel/Cavalier_king_charles_spaniel_03261.jpg'
 '/content/drive/My Drive/dogImages/train/048.Chihuahua/Chihuahua_03416.jpg']


In [7]:
print(valid_targets)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [8]:
dog_names = [item[20:-1] for item in sorted(glob("/content/drive/My Drive/dogImages2/train/*/"))]


print('There are %d training dog images.' % len(train_files))
print('There are %d validation dog images.' % len(valid_files))
print('There are %d test dog images.'% len(test_files))

There are 6680 training dog images.
There are 835 validation dog images.
There are 836 test dog images.


In [0]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [10]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 836/836 [00:08<00:00, 94.37it/s] 


In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    horizontal_flip=True) 

datagen.fit(train_tensors)

In [12]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization())


model.add(GlobalAveragePooling2D())

model.add(Dense(133, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 111, 111, 16)      64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 54, 54, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 52, 52, 64)        18496     
__________

In [0]:
model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
epochs = 10
batch_size = 20

model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=batch_size),
                    validation_data=(valid_tensors, valid_targets), 
                    steps_per_epoch=train_tensors.shape[0] // batch_size,
                    epochs=epochs, verbose=2)

Epoch 1/10
 - 81s - loss: 4.8897 - acc: 0.0174 - val_loss: 5.0586 - val_acc: 0.0048
Epoch 2/10
 - 79s - loss: 4.7056 - acc: 0.0487 - val_loss: 5.0891 - val_acc: 0.0096
Epoch 3/10
 - 79s - loss: 4.6040 - acc: 0.0645 - val_loss: 5.1080 - val_acc: 0.0108
Epoch 4/10
 - 79s - loss: 4.5319 - acc: 0.0729 - val_loss: 5.1306 - val_acc: 0.0108
Epoch 5/10
 - 79s - loss: 4.4792 - acc: 0.0795 - val_loss: 5.1343 - val_acc: 0.0108
Epoch 6/10
 - 79s - loss: 4.4269 - acc: 0.0858 - val_loss: 5.1520 - val_acc: 0.0108
Epoch 7/10
 - 79s - loss: 4.3753 - acc: 0.0961 - val_loss: 5.1731 - val_acc: 0.0096
Epoch 8/10
 - 79s - loss: 4.3365 - acc: 0.1000 - val_loss: 5.1847 - val_acc: 0.0084
Epoch 9/10
 - 79s - loss: 4.3010 - acc: 0.1087 - val_loss: 5.2045 - val_acc: 0.0096
Epoch 10/10
 - 79s - loss: 4.2698 - acc: 0.1061 - val_loss: 5.2259 - val_acc: 0.0084


In [15]:
from sklearn.metrics import confusion_matrix
# get index of predicted dog breed for each image in test set
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

cm = confusion_matrix(np.argmax(test_targets,1),np.asarray(dog_breed_predictions))

print(cm)
                      
# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

[[5 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Test accuracy: 6.9378%


In [16]:

import numpy as np

### TODO: Obtain bottleneck features from another pre-trained CNN.
bottleneck_features = np.load('/content/drive/My Drive/bottleneck_features/DogResnet50Data.npz')
train_ResNet50 = bottleneck_features['train']
valid_ResNet50 = bottleneck_features['valid']
test_ResNet50 = bottleneck_features['test']

print(train_ResNet50)

[[[[7.2806221e-01 3.8177130e-01 3.7170208e-01 ... 2.2422303e-01
    2.7094164e-01 1.9768992e-02]]]


 [[[1.0059553e+00 6.8839407e-01 0.0000000e+00 ... 2.1531639e-01
    1.1211441e+00 9.8433543e-04]]]


 [[[9.3968622e-02 1.9756782e-01 0.0000000e+00 ... 3.9581805e-01
    1.7996365e+00 1.1467970e-03]]]


 ...


 [[[2.0537768e-01 8.7052494e-02 5.7939609e-04 ... 0.0000000e+00
    5.1938808e-03 8.0556148e-01]]]


 [[[2.9411476e-02 2.5884500e-01 2.6960450e-01 ... 0.0000000e+00
    7.7637322e-02 9.3928851e-02]]]


 [[[2.5634682e+00 2.0309021e-01 1.5011416e-01 ... 3.5734051e-01
    2.7579477e-01 2.0164295e-01]]]]


In [17]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

ResNet_model = Sequential()
ResNet_model.add(GlobalAveragePooling2D(input_shape=train_ResNet50.shape[1:]))
ResNet_model.add(Dense(133, activation='softmax'))

ResNet_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 133)               272517    
Total params: 272,517
Trainable params: 272,517
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras.optimizers import Adam, Adamax

ResNet_model.compile(loss='categorical_crossentropy', optimizer=Adamax(lr=0.002), metrics=['accuracy'])

In [35]:

epochs = 10
batch_size = 64

ResNet_model.fit(train_ResNet50, train_targets, 
          validation_data=(valid_ResNet50, valid_targets),
          epochs=epochs, batch_size=batch_size, verbose=2)

Train on 6680 samples, validate on 835 samples
Epoch 1/10
 - 0s - loss: 0.0023 - acc: 0.9993 - val_loss: 15.3262 - val_acc: 0.0108
Epoch 2/10
 - 0s - loss: 0.0027 - acc: 0.9988 - val_loss: 15.3342 - val_acc: 0.0120
Epoch 3/10
 - 0s - loss: 0.0026 - acc: 0.9987 - val_loss: 15.3455 - val_acc: 0.0120
Epoch 4/10
 - 0s - loss: 0.0027 - acc: 0.9990 - val_loss: 15.3567 - val_acc: 0.0120
Epoch 5/10
 - 0s - loss: 0.0023 - acc: 0.9991 - val_loss: 15.3696 - val_acc: 0.0108
Epoch 6/10
 - 0s - loss: 0.0024 - acc: 0.9988 - val_loss: 15.3784 - val_acc: 0.0108
Epoch 7/10
 - 0s - loss: 0.0023 - acc: 0.9991 - val_loss: 15.3850 - val_acc: 0.0120
Epoch 8/10
 - 0s - loss: 0.0019 - acc: 0.9991 - val_loss: 15.3996 - val_acc: 0.0120
Epoch 9/10
 - 0s - loss: 0.0026 - acc: 0.9987 - val_loss: 15.4049 - val_acc: 0.0120
Epoch 10/10
 - 0s - loss: 0.0026 - acc: 0.9987 - val_loss: 15.4208 - val_acc: 0.0120


In [36]:
### TODO: Calculate classification accuracy on the test dataset.

# get index of predicted dog breed for each image in test set
ResNet50_predictions = [np.argmax(ResNet_model.predict(np.expand_dims(feature, axis=0))) for feature in test_ResNet50]

# report test accuracy
test_accuracy = 100*np.sum(np.array(ResNet50_predictions)==np.argmax(test_targets, axis=1))/len(ResNet50_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 71.8900%
